##주가 데이터 불러오기 모듈 설치

In [1]:
!pip install -U finance-datareader
import FinanceDataReader as fdr

Requirement already up-to-date: finance-datareader in /opt/conda/lib/python3.8/site-packages (0.9.6)


In [2]:
#df_krx = fdr.StockListing('KRX')

In [3]:
# KRX stock delisting symbol list and names 상장폐지 종목 전체 리스트
#krx_delisting = fdr.StockListing('KRX-DELISTING')

기업 코드 알아보기

In [5]:
name=input('주식 코드 기업명 ?') #find 종목코드 및 정보
df_krx[df_krx['Name'].str.contains(name)]

주식 코드 기업명 ?005930


NameError: name 'df_krx' is not defined

관심 기업 데이터 불러오기

In [6]:
삼성전자 = fdr.DataReader('005930', '2015-01-01', '2019-12-31')
현대차 = fdr.DataReader('005380', '2015-01-01', '2019-12-31')
셀트리온 = fdr.DataReader('068270', '2015-01-01', '2019-12-31')
한전 = fdr.DataReader('015760', '2015-01-01', '2019-12-31')
SK텔레컴 = fdr.DataReader('017670', '2015-01-01', '2019-12-31')

주가 종가 합치기

In [7]:
import pandas as pd
df = pd.concat([삼성전자['Close'],현대차['Close'],셀트리온['Close'],한전['Close'],SK텔레컴['Close']], axis=1)
df.columns=['삼성전자','현대차','셀트리온','한전','SK텔레컴']
df.head(3)

,삼성전자,현대차,셀트리온,한전,SK텔레컴
Date,,,,,
2015-01-02,26600,169000,35532,42700,272500
2015-01-05,26660,168000,35350,42000,276500
2015-01-06,25900,164500,34715,41850,274500


일별 수익률 계산

In [8]:
df0=df.diff(axis=0,periods=1)/df.shift(periods=1)
df0.dropna(inplace=True)
df0.head(3)

,삼성전자,현대차,셀트리온,한전,SK텔레컴
Date,,,,,
2015-01-05,0.002256,-0.005917,-0.005122,-0.016393,0.014679
2015-01-06,-0.028507,-0.020833,-0.017963,-0.003571,-0.007233
2015-01-07,0.009266,0.033435,-0.009160,0.031063,-0.018215


연, 월, 일, 분기 변수 만들기

In [0]:
import datetime
df0['요일'] = df0.index.weekday
df0['분기'] = df0.index.quarter
df0['주'] = df0.index.week
df0['연도'] = df0.index.year
df0['월'] = df0.index.month
df0.head(3)

월별 수익율 계산

In [0]:
df_rev = df0.groupby(['연도','월']).mean().iloc[:,0:5]
df_rev.head(5)

In [0]:
기대 월수익율 구하기

In [0]:
import numpy as np
rev_array=np.array(df_rev)
one = np.array(np.repeat(1,rev_array.shape[0]))
exp = one.T@rev_array/rev_array.shape[0] 
exp

2개 포트폴리오(안) 구성

In [0]:
w = np.array([[0.2, 0.2, 0.2, 0.2, 0.2],
              [0.3, 0.1, 0.2, 0.1, 0.3],
             [0.3,0.2,0.2,0.1,0.2]])

포트폴리오 기대 수익률

In [0]:
w@exp 

공분산 행렬구하기

In [0]:
#공분산 Cov(X,Y) = E[(x-mu_x)*(y-mu_y)]
exp_repeat = np.tile(exp,(rev_array.shape[0],1))
A = rev_array.T - exp_repeat.T 
B = rev_array - exp_repeat 
(A @ B)  
cov = (A @ B)/rev_array.shape[0]

포트폴리오 위험

In [0]:
w@cov@w.T

#과업 최소분산(위험) 포트폴리오 구성

In [0]:
import numpy.linalg as lin
np.array([1,1,1,1,1]).T@lin.inv(cov)
np.array([1,1,1,1,1]).T@lin.inv(cov)@np.array([1,1,1,1,1])
w = np.array((np.array([1,1,1,1,1]).T@lin.inv(cov))/(np.array([1,1,1,1,1]).T@lin.inv(cov)@np.array([1,1,1,1,1])))
print('가중치',w,'기대수익률=',w.T@exp,'위험=',w.T@cov@w)

##수비 위치에 따른 선수 연봉차이

In [0]:
import pandas as pd
salary=pd.read_csv('http://wolfpack.hnu.ac.kr/Big_Data/data/MLB1871_2018/Salaries.csv')
field=pd.read_csv('http://203.247.53.31/Big_Data/data/MLB1871_2018/Fielding.csv')
df_sal=salary[(salary.yearID==2016) & (salary.salary>0)]; df_field=field[field.yearID==2016]
df_fin=pd.merge(df_sal,df_field,on='playerID')
df_fin.head(3)

In [0]:
pd.options.display.float_format = '{:.2f}'.format
df0=df_fin.iloc[:,[2,4,9]]
m_df=df0.groupby(['POS','lgID_x']).mean()
sd_df=df0.groupby(['POS','lgID_x']).std()
q1_df=df0.groupby(['POS','lgID_x']).quantile(0.25)
q3_df=df0.groupby(['POS','lgID_x']).quantile(0.75)

In [0]:
tab_df=pd.concat([m_df,sd_df,q1_df,q3_df],axis=1)
tab_df.columns=(['평균','표준편차','1사분위','3사분위'])
tab_df

In [0]:
import seaborn as sns
sns.boxplot(x='POS', y='salary', hue='lgID_x',data=df_fin, palette="Set3")